In [2]:
from matplotlib import pylab
import nengo
import numpy as np

%matplotlib inline

In [82]:
model = nengo.Network(seed=3)
SIM_TIME = 50
SWITCH_TIME = 500
SWITCH_TIME2 = 50
SIM_TIME2 = SWITCH_TIME2*0.001
dist = nengo.dists.Uniform(-1.0, 1.0)
vals = dist.sample((1000*SIM_TIME/SWITCH_TIME)+1)
vals2 = np.linspace(-1, 1, 45)
j = []
def fn(vals):
    s = SWITCH_TIME
    vs = vals
    t = 0
    cur = 0
    i = 0
    while True:
    #initial training
        j.append(vs[i])
        t = yield [vs[i], 0, 1]
        if t > SIM_TIME:
            break
        if t-cur > 0.001*s:
            cur = t
            i += 1
    s = SWITCH_TIME2
    vs = vals2
    i = 0
    cur = t
    while True:
    #assessment 1
        j.append(vs[i])
        t = yield [vs[i], 0, 0]
        if i >= len(vals2):
            print 'too far'
            break
        if t-cur > 0.001*s:
            cur = t
            i += 1
    while True:
        t = yield [0, 0, 0]

In [103]:
func = fn(vals)
func.next()
func = func.send

In [104]:
with model:
    stim = nengo.Node(func)
    
    sensory = nengo.Ensemble(n_neurons=200, dimensions=3,
                            intercepts=nengo.dists.Uniform(0.87,0.91))
    
    
    spatial = nengo.Ensemble(n_neurons=200, dimensions=2,
                            intercepts=nengo.dists.Uniform(0.87,0.91))
    
    nengo.Connection(stim, sensory[:],
                     function=lambda x: (np.sin(x[0]*np.pi/2), 
                                         np.cos(x[0]*np.pi/2), 
                                         x[1]))
    
    #nengo.Connection(sensory[:2], sensory[2:], synapse=0.2)
    
    c = nengo.Connection(sensory, spatial, function=lambda x: [0,0],
                         learning_rule_type=nengo.PES(learning_rate=3e-4))
        
    error = nengo.Ensemble(n_neurons=100, dimensions=3)
    
    nengo.Connection(stim[2], error[2])
    nengo.Connection(sensory[:2], error[:2],
                     transform=-1)
    nengo.Connection(error, c.learning_rule, 
                     function=lambda x: x[:2] if x[2] > 0.5 else [0, 0])
    #nengo.Connection(spatial, c.learning_rule, synapse=0.1)
    
    nengo.Connection(spatial, error[:2])
    #correct = nengo.Node([0,0])
    #nengo.Connection(correct, error, transform=-1)
    
    error_probe = nengo.Probe(error)
    spatial_probe = nengo.Probe(spatial.neurons)
    spatial_probe_v = nengo.Probe(spatial)
    


In [105]:
sim = nengo.Simulator(model)
sim.run(SIM_TIME)

Simulation finished in 0:00:58.                                                 


In [106]:
def sim_gen(time, cnt):
    cur = 0
    while cur < cnt:
        yield np.mean(sim.data[spatial_probe][-SWITCH_TIME2:], axis=0)
        print sim.data[spatial_probe].shape
        sim.run(time)
        cur += 1
        
def calc_norm(rates):
    #assume that cells with similar peak centers would fall under the same
    #multicellular recording therefore peaks are done relative to all values with similar peaks
    nrm_rates = []
    rate_peaks = np.max(rates, axis=1)
    max_i = np.argmax(rates, axis=0)
    for j, i in enumerate(max_i):
        nrm_rates.append(rates[:, j]/rate_peaks[i])
    nrm_rates = np.array(nrm_rates).transpose()
    return nrm_rates

def align_cut(nrm, dist, num_checks):
    #below uses the protocol from the paper where the center value of the values with >= 50% of
    #the max value is used as the max
    #cut assumes that it is not multimodal
    cut = [[(i, neuron_val) for i, neuron_val in enumerate(nrm[:, col]) 
            if neuron_val >= 0.5] for col in range(nrm.shape[1])]
    maxes = [(i, neuron_vals[len(neuron_vals)/2][0]) 
             for i, neuron_vals in enumerate(cut) 
             if len(neuron_vals) > 0]
    max_cuts = [nrm[mx-dist:mx+dist+1, i] 
                for i, mx in maxes 
                if mx >= dist and mx <= num_checks-(dist+1)]
    return np.array(max_cuts)

def prep_graph(time, cnt, dist):
    sim_g = sim_gen(time, cnt)
    sim_g.next()
    return align_cut(calc_norm(np.array([mean for mean in sim_g])), dist)

def graph_neurons(vals, step):
    for v in range(vals.shape[0]):
        pylab.plot(range(-DIST*step, (DIST+1)*step, step), vals[v, :])

def graph_mean(mean, step):
    pylab.plot(range(-DIST*step, (DIST+1)*step, step), mean)

In [107]:
sim_g = sim_gen(SIM_TIME2, len(vals2))
a=sim_g.next()
print a.shape
mean = np.array([mean for mean in sim_g])

(200,)
(50000, 200)
Simulation finished in 0:00:01.                                                 
(50050, 200)
Simulation finished in 0:00:01.                                                 
(50100, 200)
Simulation finished in 0:00:01.                                                 
(50150, 200)
Simulation finished in 0:00:01.                                                 
(50200, 200)
Simulation finished in 0:00:01.                                                 
(50250, 200)
Simulation finished in 0:00:01.                                                 
(50300, 200)
Simulation finished in 0:00:01.                                                 
(50350, 200)
Simulation finished in 0:00:01.                                                 
(50400, 200)
Simulation finished in 0:00:01.                                                 
(50450, 200)
Simulation finished in 0:00:01.                                                 
(50500, 200)
Simulation finished in 0:00:01.         

In [88]:
n = calc_norm(mean)

In [89]:
DIST = 10
out = align_cut(n, DIST, len(vals2))

In [90]:
print mean.shape
print n.shape
print out.shape
print [len(x) for x in out]
print np.mean(mean[:, :], axis=1)

(44, 200)
(44, 200)
(50,)
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 21, 21, 21, 21, 21, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21]
[  5.7   2.9  23.9  39.1  29.9  21.2  11.4  12.9  18.1  19.1  16.3  10.7
  24.5  28.    6.7   3.9  15.3  26.7  39.2  26.3  17.    5.    8.   22.3
  19.5  18.5  41.9  56.6  53.1  44.8  29.9  26.7  26.2  27.9  48.3  42.7
  18.8  16.2  21.3  25.   29.9  29.2  18.5  19.2]


In [17]:

#vals = prep_graph(SIM_TIME2, len(vals2), DIST)
mean_vals = np.mean(vals[:, :], axis=0)

Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:01.                                                 
Simulation finished in 0:00:

IndexError: too many indices for array

In [101]:
print j[-(50*46):-(50*44)]

[-0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.92836754748143546, -0.928367

In [18]:
print vals.shape

(31,)


In [ ]:
STEP = 10

In [15]:
graph_neurons(vals, STEP)

(50, 200)


In [ ]:
graph_mean(mean_vals, STEP)